In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

In [ ]:
#Following is the code to load required libraries and data:
import numpy as np
import pandas as pd

train_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)

test_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test' , skiprows = 1, header = None)

col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num','marital_status', 'occupation','relationship', 'race', 'sex', 'capital_gain','capital_loss', 'hours_per_week', 'native_country', 'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

In [ ]:
df=pd.concat([train_set,test_set],axis=0)

In [ ]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'wage_class'],
      dtype='object')

In [ ]:
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
wage_class        0
dtype: int64

In [ ]:
df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
wage_class        object
dtype: object

In [ ]:
df.describe()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [ ]:
cat=[]
for i in df.columns:
    if(df[i].dtype=='object'):
        cat.append(i)
        
cat

['workclass',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native_country',
 'wage_class']

In [ ]:
df[cat].describe()

,workclass,education,marital_status,occupation,relationship,race,sex,native_country,wage_class
count,48842,48842,48842,48842,48842,48842,48842,48842,48842
unique,9,16,7,15,6,5,2,42,4
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,33906,15784,22379,6172,19716,41762,32650,43832,24720


In [ ]:
for i in cat:
    print('-'*20)
    print(i)
    print('-'*20)
    print(df[i].value_counts())
    

--------------------
workclass
--------------------
 Private             33906
 Self-emp-not-inc     3862
 Local-gov            3136
 ?                    2799
 State-gov            1981
 Self-emp-inc         1695
 Federal-gov          1432
 Without-pay            21
 Never-worked           10
Name: workclass, dtype: int64
--------------------
education
--------------------
 HS-grad         15784
 Some-college    10878
 Bachelors        8025
 Masters          2657
 Assoc-voc        2061
 11th             1812
 Assoc-acdm       1601
 10th             1389
 7th-8th           955
 Prof-school       834
 9th               756
 12th              657
 Doctorate         594
 5th-6th           509
 1st-4th           247
 Preschool          83
Name: education, dtype: int64
--------------------
marital_status
--------------------
 Married-civ-spouse       22379
 Never-married            16117
 Divorced                  6633
 Separated                 1530
 Widowed                   1518
 Married

In [ ]:

#removing "?" from workclass,occupation,native_country


In [ ]:

print("Total shape before deleting ")
print(df.shape)
ind=df[df['workclass']==" ?"].index
df.drop(ind,axis=0,inplace=True)
print("shape after deleting  workclass")
print(df.shape)
ind1=df[df['occupation']==" ?"].index
df.drop(ind1,axis=0,inplace=True)
print("shape after deleting  occupation")
print(df.shape)
ind2=df[df['native_country']==" ?"].index
df.drop(ind2,axis=0,inplace=True)
print("shape after deleting  native_country")
print(df.shape)

Total shape before deleting 
(48842, 15)
shape after deleting  workclass
(44260, 15)
shape after deleting  occupation
(44244, 15)
shape after deleting  native_country
(42960, 15)


In [ ]:
#checking " ?" values in dataset
for i in cat:
    print('-'*20)
    print(i)
    print('-'*20)
    print(df[i].value_counts())
    

--------------------
workclass
--------------------
 Private             31661
 Self-emp-not-inc     3609
 Local-gov            2949
 State-gov            1837
 Self-emp-inc         1568
 Federal-gov          1318
 Without-pay            18
Name: workclass, dtype: int64
--------------------
education
--------------------
 HS-grad         14042
 Some-college     9430
 Bachelors        7181
 Masters          2382
 Assoc-voc        1856
 11th             1544
 Assoc-acdm       1432
 10th             1166
 7th-8th           780
 Prof-school       732
 9th               638
 12th              554
 Doctorate         515
 5th-6th           427
 1st-4th           213
 Preschool          68
Name: education, dtype: int64
--------------------
marital_status
--------------------
 Married-civ-spouse       20024
 Never-married            13868
 Divorced                  5997
 Separated                 1321
 Widowed                   1198
 Married-spouse-absent      520
 Married-AF-spouse           3

In [ ]:
cat

['workclass',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native_country',
 'wage_class']

In [ ]:
df1=pd.get_dummies(df[cat[:-1]],drop_first=True)
df1.shape

(42960, 90)

In [ ]:
int_col=[]
for i in df.columns:
    if(df[i].dtype!='object'):
        int_col.append(i)
int_col
df_int=df[int_col]
df_int.shape

(42960, 6)

In [ ]:
X=pd.concat([df_int,df1],axis=1) 
X.shape

(42960, 96)

In [ ]:
#CONVERTING the datatype target variable from categorical 

In [ ]:
Y=df.wage_class

In [ ]:

he=LabelEncoder()
Y=he.fit_transform(Y)
#data = np.array(he.fit_transform(df.wage_class), dtype = np.str)
Y=pd.DataFrame(Y)

In [ ]:

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=1,test_size=0.2)

###Applying XGB##

In [ ]:
xgb=XGBClassifier(objective='binary:logistic')
xgb.fit(X_train,Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
pred=xgb.predict(X_test)
accuracy_score(Y_test,pred)

0.5778631284916201

In [ ]:
params = {
    'objective': 'binary:logistic',
    'max_depth': 2,
    'learning_rate': .5,
    'silent': True,    # would be boolean in sklearn
    'n_estimators': 100
}

xgb1=XGBClassifier(**params)
xgb1.fit(X_train,Y_train)
pred=xgb1.predict(X_test)
accuracy_score(Y_test,pred)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5857774674115456

In [ ]:
#USING GRID SEARCH CV
params={
        'max_depth': range(1,5,1),
  'learning_rate' : [0.1,0.01,0.001],
    'n_estimators' : range(10,100,10)
   
}
grid_search=GridSearchCV(estimator=XGBClassifier(objective='binary:logistic'),cv=5,n_jobs=-1,param_grid=params,verbose=6)
grid_search.fit(X_train,Y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 24.4min
[Parallel(n_jobs=-1)]: Done 309 tasks      | elapsed: 37.5min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 55.2min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 71.4min finished
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01, 0.001],
                         'max_depth': range(1, 5),
                         'n_estimators': range(10, 100, 10)},
    

In [29]:
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 90}

In [30]:
xgb2=XGBClassifier(objective='logistic:binary',learning_rate=0.1,max_depth=4,n_estimators=90)
xgb2.fit(X_train,Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=90, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [31]:
pred2=xgb2.predict(X_test)
accuracy_score(Y_test,pred)

0.5857774674115456